<a href="https://colab.research.google.com/github/vladargunov/CreditDerivativesSimulator/blob/master/templates/notebooks/template_notebook_v1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\Huge Version 1.2$

# Template for Developing Credit Trading Strategies

What's new in version 1.2?

- Thanks to [ArturArifullin](https://github.com/ArturArifullin) and [YaFandCo](https://github.com/YaFandCo) the Sharpe and Annualised Returns are computed correctly.

- Now your portfolio weights are reported in Wandb.

- The Sharpe is now dynamic and is reported for the last year and the whole period separately.

- From this version the simulator will compute two sets of metrics, first involving transaction costs and the other one without them.

- Added steps to dates table to find the date periods in hte graphs.

- If you submit your strategy into "Final" project, the date for test period will be automatically set to 2019-01-02. For "Test" projects you can set any convenient for you date.



Please set a version of simulator that you require, it is very advisable to use the latest version with tag "latest" since it includes all latest features.

In [1]:
VERSION =  "latest" #@param {type:"string"}

In [7]:
#@title [RUN] Install required python libraries and clone the repo

# Assert for correct version
assert VERSION in ['v1.0', 'v1.1', 'v1.2', 'latest'], 'Specify correct version!' + \
                            'Available versions are "v1.0", "v1.1" , "v1.2" and "latest".'

!pip install wandb -q
if VERSION == "latest":
    !git clone https://github.com/vladargunov/CreditDerivativesSimulator.git
else:
    !git clone -b $VERSION https://github.com/vladargunov/CreditDerivativesSimulator.git

# Import necessary classses
%cd CreditDerivativesSimulator
from src.base_strategy import BaseStrategy
from src.simulator import Simulator

Cloning into 'CreditDerivativesSimulator'...
remote: Enumerating objects: 257, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 257 (delta 47), reused 40 (delta 23), pack-reused 188
Receiving objects: 100% (257/257), 49.46 KiB | 1.15 MiB/s, done.
Resolving deltas: 100% (161/161), done.
/content/CreditDerivativesSimulator/CreditDerivativesSimulator


# Migration from previous version

As in previous versions, all you need to do is to copy your strategy in the next cell.

In [3]:
class MyFirstStrategy(BaseStrategy):
  def train_model(self, train_data):
    pass

  def trade(self, daily_data) -> dict:
    return {'spx' : .1}

# Testing your strategy

Note that a simulator now requires an additional parameter of *transaction_costs*, which computes the fraction of portfolio values that were changed from the previous day. For example, if your portfolio value in the previous day was 1.5 and you increased your position on 0.1 value, then the transaction cost for such operation would be 

$ Cost = 1.5 * 0.1 * TransactionCosts$, 

which will be subtracted from portfolio value and will change your daily return and other metrics.

In [ ]:
# Create an instance of your strategy and of simulator
my_strategy = MyFirstStrategy()


# If you use wandb, you will need to past an API key from your wandb account
sim = Simulator(use_wandb=True, 
                debug_mode=False, 
                train_test_split_time='2019-01-02', # submit date in format 'yyyy-mm-dd'
                transaction_costs=0.003, # Fraction of changes in portfolio that goes into transaction costs
                run_name='MyFirstStrategy1',
                project_name='Test') # or 'Final'

In [ ]:
# Available tickers in new version
sim.get_available_assets()

In [ ]:
# Simulate your strategy
sim.simulate(strategy=my_strategy)

# Correct calculations of Sharpe and Annualised Return

As it was correctly pointed out to me, the correct formulas for Sharpe and Annualised return are

$$ Sharpe = \frac{average \; daily \; return - risk \; free \;rate}{standard \; deviation \; daily \; return} * \sqrt{252}$$

$$ Annualised \; return = [(1 + daily \; return_1) * (1 + daily \; return_2) * ... * (1 + daily \; return_n) ]^{252 / n} - 1 $$

# Find dates corresponding to the test metric "Step" in the simulator

Since we need to observe how portfolio behaves over time and know correctly at which date it happens, now the simulator produces tables of dates that correspond to metric "Step" from simulator. 

If you work in "Final" environment, this table is available directly in Wandb UI.

Additionally, if you work in "Test" environment, the csv file is available in the path "/content/CreditDerivativesSimulator/supplementary_data".

Unfortunately, the dates as x-metrics are not supprted by Wandb, but the things might change in the nearest future, see this [issue](https://github.com/wandb/wandb/issues/4063).

# Pro Tips

In this last section I will introduce some tricks that may make your development of strategies easier.

## Pro Tip 1:

Set you Wandb API in the notebook to not to copy each time you want to test your strategy, this is done by one-liner below:

In [ ]:
import os
os.environ["WANDB_API_KEY"] = 'YOUR WANDB API KEY'

## Pro Tip 2:

If you wish to log your custom metrics wich are not presented in your strategy, you can do it by the simple command 

**wandb.log({"name_metric" : value_metric}, commit=False)**

Please make sure that argument **commit** is set to **False**, as otherwise the simulator might not work correctly.

Then please contact me about what you wish to introduce and I will add your metric to the Wandb UI.

See code below on example how it can be achieved:

In [ ]:
import random
import wandb

class MyFirstStrategy(BaseStrategy):
  def train_model(self, train_data):
    pass

  def trade(self, daily_data) -> dict:
    some_metric = random.rand()
    # Log your custom metric each trade day
    wandb.log({'some_metric' : some_metric})
    return {'spx' : .1}